In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
%cd gdrive/My Drive/github_project
! git clone https://github.com/TobiasSkovgaardJepsen/posts.git


/content/gdrive/My Drive/github_project
Cloning into 'posts'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 26 (delta 4), reused 0 (delta 0), pack-reused 11
Unpacking objects: 100% (26/26), done.


In [ ]:
%cd /content/gdrive/My Drive/github_project/TF-GNN

/content/gdrive/My Drive/github_project/TF-GNN


In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
# coding=utf-8
from gnn.data.dataset import GraphDataset, WhiteSpaceTokenizer
from gnn.data.example import load_M10, load_cora, load_dblp
from gnn.model.gcn import GCN, GCNTrainer
import tensorflow as tf

# eager mode must be enabled
from tensorflow.contrib.eager.python import tfe

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"



In [ ]:
tfe.enable_eager_execution()

# read graph dataset: data/M10 data/dblp
# dataset = GraphDataset("data/dblp", ignore_featureless_node=True)
dataset = load_M10("data/M10", ignore_featureless_node=True)



In [ ]:
type(dataset)

gnn.data.dataset.GraphDataset

In [ ]:
adj = dataset.adj_matrix(sparse=True)
feature_matrix, feature_masks = dataset.feature_matrix(bag_of_words=True, sparse=True)
labels, label_masks = dataset.label_list_or_matrix(one_hot=False)

train_node_indices, test_node_indices, train_masks, test_masks = dataset.split_train_and_test(training_rate=0.3)

gcn_model = GCN([16, dataset.num_classes()], drop_rate=0.1)
gcn_trainer = GCNTrainer(gcn_model)
gcn_trainer.train(adj, feature_matrix, labels, train_masks, test_masks, learning_rate=1e-3, l2_coe=1e-3)

/content/gdrive/My Drive/github_project/TF-GNN/gnn/model/gcn.py:79: RuntimeWarning: divide by zero encountered in power
  inv_D = np.power(inv_D, -0.5)



Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

step = 0	loss = 2.3213677406311035
step = 0	accuracy = 0.16724400720520993	macro_f1 = 0.1291682314000006	micro_f1 = 0.16724400720520993
step = 20	loss = 1.837527871131897
step = 20	accuracy = 0.6511015657475405	macro_f1 = 0.588973718822067	micro_f1 = 0.6511015657475405
step = 40	loss = 1.4535194635391235
step = 40	accuracy = 0.7224608563114868	macro_f1 = 0.6582693739919635	micro_f1 = 0.7224608563114869
step = 60	loss = 1.2063698768615723
step = 60	accuracy = 0.7546071774975752	macro_f1 = 0.6893655303375346	micro_f1 = 0.7546071774975751
step = 80	loss = 1.0335527658462524
step = 80	accuracy = 0.7659692392961064	macro_f1 = 0.6972840195389869	micro_f1 = 0.7659692392961064
step = 100	loss = 0.9139356017112732
step = 100	accuracy = 0.7789940418456422	macro_f1 = 0.7142520314284952	micro_f1 = 0.778994041

# Numpy simple GCN

![alt text](https://miro.medium.com/max/864/1*jTW7doI_cqC_p9XQrmuu9A.png)

In [ ]:
import numpy as np
import tensorflow as tf


In [ ]:
A = np.matrix([   #邻接矩阵
    [0, 1, 0, 0],
    [0, 0, 1, 1], 
    [0, 1, 0, 0],
    [1, 0, 1, 0]],
    dtype=float
)

In [ ]:
X = np.matrix([   #X为输入特征， 这里是由A生成的
            [i, -i]
            for i in range(A.shape[0])
        ], dtype=float)
print(A)
print(X)        


[[0. 1. 0. 0.]
 [0. 0. 1. 1.]
 [0. 1. 0. 0.]
 [1. 0. 1. 0.]]
[[ 0.  0.]
 [ 1. -1.]
 [ 2. -2.]
 [ 3. -3.]]


In [ ]:
I = np.matrix(np.eye(A.shape[0])) #单位矩阵
print(I)

[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]


In [ ]:
D = np.array(np.sum(A, axis=0))[0]  #D为度矩阵 degree matrix 先计算A中每一个node与其他node的连接数量
D = np.matrix(np.diag(D)) #再将上面的值作为对角线生成矩阵
print(D)

[[1. 0. 0. 0.]
 [0. 2. 0. 0.]
 [0. 0. 2. 0.]
 [0. 0. 0. 1.]]


传播规则为f(X, A) = D⁻¹AX



A_hat = A + I

In [ ]:
D_t = D**-1
print(D_t)

[[1.  0.  0.  0. ]
 [0.  0.5 0.  0. ]
 [0.  0.  0.5 0. ]
 [0.  0.  0.  1. ]]


In [ ]:
A_hat = A + I
print(A_hat)

[[1. 1. 0. 0.]
 [0. 1. 1. 1.]
 [0. 1. 1. 0.]
 [1. 0. 1. 1.]]


In [ ]:
print(D_t*A)

[[0.  1.  0.  0. ]
 [0.  0.  0.5 0.5]
 [0.  0.5 0.  0. ]
 [1.  0.  1.  0. ]]


In [ ]:
f = D_t*A*X
print(f)

[[ 1.  -1. ]
 [ 2.5 -2.5]
 [ 0.5 -0.5]
 [ 2.  -2. ]]


In [ ]:
D_hat = np.array(np.sum(A_hat, axis=0))[0]  #D为度矩阵 degree matrix 先计算A_hat中每一个node与其他node的连接数量
D_hat = np.matrix(np.diag(D_hat)) #再将上面的值作为对角线生成矩阵
print(D_hat)

[[2. 0. 0. 0.]
 [0. 3. 0. 0.]
 [0. 0. 3. 0.]
 [0. 0. 0. 2.]]


In [ ]:
W = np.matrix([   #初始权重矩阵  
             [1, -1],
             [-1, 1]
         ])
# 如果想降维度，那就将w的size降维度
# W = np.matrix([
#              [1],
#              [-1]
#          ])

In [ ]:
f_hat = D_hat**-1 * A_hat * X * W 
print(f_hat)

[[ 1. -1.]
 [ 4. -4.]
 [ 2. -2.]
 [ 5. -5.]]


In [ ]:
tf.nn.relu(f_hat) #负值全部变零，正值不变

<tf.Tensor: shape=(4, 2), dtype=float64, numpy=
array([[1., 0.],
       [4., 0.],
       [2., 0.],
       [5., 0.]])>

In [ ]:
from networkx import karate_club_graph, to_numpy_matrix

In [ ]:
zkc = karate_club_graph()
order = sorted(list(zkc.nodes())) #34个node 按照0-33 排列
print(order)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33]


In [ ]:
A = to_numpy_matrix(zkc, nodelist=order) #变成 34*34 的邻接矩阵
I = np.eye(zkc.number_of_nodes()) #生成对应的单位矩阵34*34
# print(A)
# print(zkc.number_of_nodes())
A_hat = A + I 
# 生成对应的度矩阵
D_hat = np.array(np.sum(A_hat, axis=0))[0]
D_hat = np.matrix(np.diag(D_hat))
# print(D_hat)
# print(A_hat)

In [ ]:
# 初始化随机生成两层权重数值
W_1 = np.random.normal(  # Drawn samples from the parameterized normal distribution.返回形状为（34，4）的参数化的正态分布值
    loc=0, scale=1, size=(zkc.number_of_nodes(), 4))
print(W_1.shape)
W_2 = np.random.normal(
    loc=0, size=(W_1.shape[1], 2)) #生成（4,2）形状的正态分布矩阵
print(W_2.shape)

(34, 4)
(4, 2)


In [ ]:
def gcn_layer(A_hat, D_hat, X, W):
    return tf.nn.relu(D_hat**-1 * A_hat * X * W) 

In [ ]:
H_1 = gcn_layer(A_hat, D_hat, I, W_1)
print(H_1.shape)
# H_2 = gcn_layer(A_hat, D_hat, H_1, W_2)

(34, 3)


In [ ]:
print(A_hat.shape)
print(D_hat.shape)
print(H_1.shape)
print(W_2.shape)
H_2 = gcn_layer(A_hat, D_hat, H_1, W_2)

(34, 34)
(34, 34)
(34, 3)
(3, 2)


InvalidArgumentError: ignored

In [ ]:
feature_representations = {
    node: np.array(output)[node] 
    for node in zkc.nodes()}